# Commands and State Synchronization in Narupa

This notebook explores the concepts of commands and state synchronization in Narupa. You'll learn

* How to set up custom commands on a Narupa server. 
* How to set shared variables and objects between clients via a Narupa server.

## Commands

### What is a command?

In Narupa, a *command* is a function on the server that can be run by a client. These can be used to customize a server,
or create entirely new applications. They are used in lots of places. For example, telling the simulation to pause, play, reset and step uses them! 

Let's create a Narupa server. All servers and applications that derive from a NarupaServer have the ability to run commands.

In [1]:
from narupa.core import NarupaServer

In [2]:
server = NarupaServer(address='localhost', port=0)

### A Basic Command

Let's set up a very simple command

In [3]:
def hello_world():
    print('Hello World!')

To register it, we have to provide a unique name for the command, which will be used by clients

In [4]:
server.register_command('hello', hello_world)

Now let's connect a client to the server.

In [5]:
from narupa.core import NarupaClient

In [6]:
client = NarupaClient.establish_channel(address=server.address, port=server.port)

We can ask what commands are available.

In [7]:
client.update_available_commands()

{'hello': <narupa.command.command_info.CommandInfo at 0x159bd12f548>}

Let's run it! 

In [8]:
client.run_command('hello')

Hello World!


{}

You'll notice that the output of the command was printed in the cell, and it returned an empty dictionary. This leads us nicely onto more advanced commands

### Taking Arguments and Returning Results

A command is expected to have the following signature

In [9]:
from typing import Dict
def command(**kwargs) -> Dict[str, object]:
    pass

So this means it can define keyword arguments, and can return a dictionary of objects with string keys. For practical reasons of transmitting over the wire, the dictionary must only contain simple objects, such as numbers, booleans, strings and lists and dictionaries of these things. This is because it is internally converted to a protobuf Struct, which is similar to a JSON file.

Let's make a more complicated example

In [10]:
import math

In [11]:
def pythagoras(a=1, b=1):
    c = math.sqrt(a**2 + b**2)
    return {'c': c}

In [12]:
server.register_command('pythagoras', pythagoras)

Let's run the command again - we could update the commands on the client, but we know it's registered

In [13]:
client.run_command('pythagoras')

{'c': 1.4142135623730951}

This time, we receive a dictionary of results, as defined in the method. Let's set some arguments:

In [14]:
client.run_command('pythagoras', a=3, b=4)

{'c': 5.0}

Verifying this result is left as an exercise. What happens if we send incorrect arguments?

In [15]:
import grpc
try:
    client.run_command('pythagoras', c=2)
except grpc.RpcError as e:
    print(e.details())

Exception calling application: pythagoras() got an unexpected keyword argument 'c'


Cool! Even if the client was in a different language, on a different computer, we can see why we messed up!

When registering the command, you can optionally define some defaults. Let's remove our previous definition, and add it with some defaults. This can be useful if you want to set default behaviour for some method that isn't directly under your control

In [16]:
server.unregister_command('pythagoras')
server.register_command('pythagoras', pythagoras, {'a':2,'b':2})

In [17]:
client.run_command('pythagoras')

{'c': 2.8284271247461903}

It also gives the client some information about what the command accepts:

In [18]:
client.update_available_commands()
client.available_commands['pythagoras'].arguments

{'a': 2.0, 'b': 2.0}

In [19]:
client.close()
server.close()

### Using commands to drive an application

Commands become powerful when they are used to *change* something on the server, making the server interactive. 

The [trajectory viewer](../mdanalysis/mdanalysis_trajectory.ipynb) example demonstrates this. 

## Synchronising State

Narupa uses a client-server model, which means clients do not talk to eachother directly. 

Howerver, it is often important to synchronise information across multiple clients, for example the position of the simulation in the VR space, or the current state of one's avatar (e.g. if you're in a menu).

We achieve this with a shared state dictionary that clients can update. Whenever a client makes a change to this dictionary, the changes are sent to any clients that are listening. 

In [20]:
server = NarupaServer(address='localhost', port=0)

In [21]:
client = NarupaClient.establish_channel(address='localhost', port=server.port)

The following command makes the client listen to any changes on the shared state dictionary.

In [22]:
client.subscribe_all_state_updates()

At first, this dictionary is empty:

In [23]:
client.copy_state()

{}

Let's connect a second client

In [24]:
second_client = NarupaClient.establish_channel(address='localhost', port=server.port)

In [25]:
second_client.subscribe_all_state_updates()

Now, the first client will make a change to the dictionary, and we'll see both clients will get the update.

In [26]:
from narupa.state.state_dictionary import DictionaryChange

We can add or update keys, and we can remove a list of keys. In what follows, we set a key `a` to the value 2, and do not remove any keys (since there aren't any)

In [27]:
updates = {'a': 2}
key_removals = []

In [28]:
changes = DictionaryChange(updates=updates,removals=key_removals)

In [29]:
client.attempt_update_state(changes)

True

Now we copy the current state for both clients, and see that both have been updated with the new value

In [30]:
client.copy_state()

{'a': 2.0}

In [31]:
second_client.copy_state()

{'a': 2.0}

The DictionaryChange object took a second argument - keys to remove. We can use that to remove things from the shared state.

In [32]:
changes = DictionaryChange({}, ['a'])
second_client.attempt_update_state(changes)

True

Now, the dictionary is empty. 

In [33]:
client.copy_state()

{}

In [34]:
second_client.copy_state()

{}

What if both clients try to update at the same time, or what if I don't want someone else to mess with something?
We handle this by locking the key while changes are made, so only one client can edit fields at a time:

In [35]:
# We aquire a lock on the key 'a' for 10 seconds.
got_lock = client.attempt_update_locks({'a': 10})
print(f'Client has a lock: {got_lock}')
successfully_updated = second_client.attempt_update_state(changes)
print(f'Second client updated state: {successfully_updated}')

Client has a lock: True
Second client updated state: False


The dictionary accepts anything that can be represented as a protobuf Struct, so you can set up complicated things:

In [36]:
changes = DictionaryChange({
    'party':{'pokemon':['charmander','squirtle','bulbasaur','pikachu']},
    'battle':{
        'name':'charmander',
        'type':'fire',
        'level': 7,
        'is_my_favourite': True,
        'abilities':['scratch', 'growl']
    }
}, [])
second_client.attempt_update_state(changes)

True

In [37]:
import pprint # pretty print!
pprint.pprint(client.copy_state())

{'battle': {'abilities': ['scratch', 'growl'],
            'is_my_favourite': True,
            'level': 7.0,
            'name': 'charmander',
            'type': 'fire'},
 'party': {'pokemon': ['charmander', 'squirtle', 'bulbasaur', 'pikachu']}}


## Tidying Up

In [38]:
client.close()
second_client.close()
server.close()

# Next Steps

* See practical examples of commands in the [trajectory viewer](../mdanalysis/trajectory_viewer.ipynb)
* Learn more about how [servers](servers.ipynb) are constructed. 
* Look at the [C# client code](https://gitlab.com/intangiblerealities/narupa-applications/narupa-imd/-/blob/master/Assets/Plugins/Narupa/Grpc/GrpcClient.cs) to learn how to run commands from a VR application.